In [90]:
# !pip install --user --upgrade tensorflow==1.8

In [91]:
import tensorflow as tf
import pandas as pd
import calendar
import time
import numpy as np

In [92]:
tf.VERSION

'1.4.0-rc1'

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, Imputer

In [94]:
books = pd.read_csv(r"../Dataset/Final/BX-Books.csv")
ratings = pd.read_csv(r"../Dataset/Final/BX-Book-Ratings.csv")
users = pd.read_csv(r"../Dataset/Final/BX-Users.csv")

/usr/local/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [95]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [96]:
users.head()

,User-ID,Age,City,State,Country
0,1,20.0,Airmont,New York,United States
1,2,18.0,Stockton,California,United States
2,3,21.0,Moscow,Moscow,Russia
3,4,17.0,Porto,Norte,Portugal
4,5,23.0,Farnborough,England,United Kingdom


In [97]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,g_year-of-publication,g_categories,g_processed
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2003,Social Science,1
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,2001,Actresses,1
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,1983,1940-1949,1
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,1999,Medical,1
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,1999,Design,1


In [98]:
full_data_df = ratings.merge(books,on='ISBN').merge(users,on='User-ID')

In [99]:
full_data_df.count()

User-ID                  1031173
ISBN                     1031173
Book-Rating              1031173
Book-Title               1031173
Book-Author              1031172
Year-Of-Publication      1031173
Publisher                1031171
Image-URL-S              1031173
Image-URL-M              1031173
Image-URL-L              1031173
g_year-of-publication    1031173
g_categories             1031173
g_processed              1031173
Age                      1031173
City                     1031173
State                    1031173
Country                  1031173
dtype: int64

In [100]:
useful_data_df = full_data_df[['City', 'State', 'Country', 'Age', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Book-Rating']].dropna()

In [101]:
useful_data_df.head()

,City,State,Country,Age,Book-Author,Year-Of-Publication,Publisher,Book-Rating
0,Tyler,Texas,United States,21.0,M. J. Rose,2002,Ballantine Books,0
1,Cincinnati,Ohio,United States,23.0,M. J. Rose,2002,Ballantine Books,5
2,Cincinnati,Ohio,United States,23.0,Orson Scott Card,1986,Tor Books,9
3,Cincinnati,Ohio,United States,23.0,TRUMAN CAPOTE,1994,Vintage,8
4,Cincinnati,Ohio,United States,23.0,Rebecca Wells,1996,HarperCollins,9


In [102]:
def getLabelEncodedColumn(col, df):
    le = LabelEncoder()
    return le.fit_transform(df[col].values).reshape(-1,1)

In [103]:
useful_data_df.count()

City                   1031170
State                  1031170
Country                1031170
Age                    1031170
Book-Author            1031170
Year-Of-Publication    1031170
Publisher              1031170
Book-Rating            1031170
dtype: int64

In [104]:
useful_data_df['Book-Author-Enc'] = getLabelEncodedColumn('Book-Author', useful_data_df)
useful_data_df['City-Enc'] = getLabelEncodedColumn('City', useful_data_df)
useful_data_df['State-Enc'] = getLabelEncodedColumn('State', useful_data_df)
useful_data_df['Country-Enc'] = getLabelEncodedColumn('Country', useful_data_df)
useful_data_df['Publisher-Enc'] = getLabelEncodedColumn('Publisher', useful_data_df)

In [16]:
useful_data_df.head()

,City,State,Country,Age,Book-Author,Year-Of-Publication,Publisher,Book-Rating,Book-Author-Enc,City-Enc,State-Enc,Country-Enc,Publisher-Enc
0,Tyler,Texas,United States,21.0,M. J. Rose,2002,Ballantine Books,0,61693,6860,927,157,1391
1,Cincinnati,Ohio,United States,23.0,M. J. Rose,2002,Ballantine Books,5,61693,1526,686,157,1391
2,Cincinnati,Ohio,United States,23.0,Orson Scott Card,1986,Tor Books,9,73674,1526,686,157,14901
3,Cincinnati,Ohio,United States,23.0,TRUMAN CAPOTE,1994,Vintage,8,93471,1526,686,157,15650
4,Cincinnati,Ohio,United States,23.0,Rebecca Wells,1996,HarperCollins,9,80510,1526,686,157,6614


In [140]:
training_df = useful_data_df.filter(['Age', 'Year-Of-Publication','Publisher-Enc',
                                   'Book-Author-Enc', 'City-Enc','State-Enc', 'Book-Rating']).head(7000)

In [141]:
X, y = training_df.drop('Book-Rating', axis=1), training_df[['Book-Rating']]

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [143]:
def getTfDataset(X, y):
    features = ['Age', 'Year-Of-Publication','Publisher-Enc',
                                   'Book-Author-Enc', 'City-Enc','State-Enc']
    target = 'Book-Rating'
    dataset = tf.data.Dataset.from_tensor_slices(
                (
                    tf.cast(X.values, tf.float32),
                    tf.cast(y.values, tf.float32)
                )
            )        
    return dataset

In [144]:
from sklearn.preprocessing import OneHotEncoder
def getOneHotEncoding(column):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit_transform(useful_data_df[column].values.reshape(-1,  1))
    return enc

In [145]:
def oneHotCodedSparseMatrix():
    return {
        'author': getOneHotEncoding('Book-Author-Enc'),
        'city': getOneHotEncoding('City-Enc'),
        'state': getOneHotEncoding('State-Enc')
    }


In [146]:
def embeddingsInputLen():
    ohcsm = oneHotCodedSparseMatrix()
    return {
        'author': len(ohcsm['author'].active_features_),
        'city': len(ohcsm['city'].active_features_),
        'state': len(ohcsm['state'].active_features_)
    }

In [147]:
embeddingsLength = embeddingsInputLen()
embeddingsLength

{'author': 101590, 'city': 7574, 'state': 1065}

In [148]:
RUN_NAME = calendar.timegm(time.gmtime())

learning_rate = 0.001
training_epochs = 30

number_of_inputs = 5
number_of_outputs = 1

layer_1_nodes = 5
layer_2_nodes = 25
layer_3_nodes = 12

In [149]:
def getInputLayers(name):
    return tf.placeholder(tf.float64, shape=(None, 1), name=name)

### Iterator

In [150]:
tf.reset_default_graph()
batchSize = 256
batchedTrainingDataset = getTfDataset(X_train, y_train).batch(batchSize)
batchedTestDataset = getTfDataset(X_test, y_test).batch(batchSize)
train_iterator = batchedTrainingDataset.make_initializable_iterator()
test_iterator = batchedTestDataset.make_initializable_iterator()

### Input Layer: Age, City, State, Author, Year of Publish

In [151]:
input_age = getInputLayers('input_age')
input_state = getInputLayers('input_state') 
input_author = getInputLayers('input_author')
input_city = getInputLayers('input_city')
input_year = getInputLayers('input_year')

### Embeddings: City, State, Author

In [152]:
weights_city = tf.get_variable(name="city_emb_wt", shape=[embeddingsLength['city'], 2], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
encoded_city_ohv = tf.one_hot(tf.cast(input_city, tf.int64), embeddingsLength['city'])
encoded_city = tf.reshape(encoded_city_ohv,(-1, embeddingsLength['city']))
embedding_city_output = tf.nn.relu(tf.matmul(tf.cast(encoded_city, tf.float64), weights_city))

In [153]:
weights_state = tf.get_variable(name="state_emb_wt", shape=[embeddingsLength['state'], 2], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
encoded_state_ohv = tf.one_hot(tf.cast(input_state, tf.int64), embeddingsLength['state'])
encoded_state = tf.reshape(encoded_state_ohv,(-1, embeddingsLength['state']))
embedding_state_output = tf.nn.relu(tf.matmul(tf.cast(encoded_state, tf.float64), weights_state))

In [154]:
weights_author = tf.get_variable(name="author_emb_wt", shape=[embeddingsLength['author'], 2], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
encoded_author_ohv = tf.one_hot(tf.cast(input_author, tf.int64), embeddingsLength['author'])
encoded_author = tf.reshape(encoded_author_ohv,(-1, embeddingsLength['author']))
embedding_author_output = tf.nn.relu(tf.matmul(tf.cast(encoded_author, tf.float64), weights_author))

### Layer 1

In [155]:
input_x = tf.concat([input_age, embedding_city_output, embedding_state_output, embedding_author_output, input_year], axis=1)
weights_1 = tf.get_variable(name="weights1", shape=[8, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_1 = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer(), dtype=tf.float64)
layer_1_output = tf.nn.relu(tf.matmul(input_x, weights_1) + biases_1)

### Layer 2

In [156]:
weights_2 = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_2 = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer(), dtype=tf.float64)
layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights_2) + biases_2)

### Layer 3

In [157]:
weights_3 = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_3 = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer(), dtype=tf.float64)
layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights_3) + biases_3)

### Output

In [158]:
weights_o = tf.get_variable("weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_o = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer(), dtype=tf.float64)
prediction = tf.matmul(layer_3_output, weights_o) + biases_o

### Cost

In [159]:
actual_y = tf.placeholder(tf.float64, shape=(None, 1), name="Actual_Y")
cost = tf.reduce_mean(tf.squared_difference(prediction, actual_y))

### Optimizer

In [160]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Logging

In [161]:
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

In [162]:
def getFeedDict(batch, batchSize, sess):
    features = batch[0].eval(session=sess)
    age = features[:,0].reshape(-1, 1)
    yop = features[:,1].reshape(-1, 1)
    pub = features[:,2].reshape(-1, 1)
    atr = features[:,3].reshape(-1, 1)
    city = features[:,4].reshape(-1, 1)
    state = features[:,5].reshape(-1, 1)
    output = batch[1].eval(session=sess)
    actualBatchSize = features.shape[0]
    print('Required Batch Size: {}, Actual Batch Size: age{}, yop{}, atr{}, city{}, state{}, output{}'.format(batchSize, age.shape, yop.shape, atr.shape, city.shape, state.shape, output.shape))
    if batchSize > actualBatchSize:
        shape = [batchSize - actualBatchSize, 1]
        age = padArrayWithZeros(shape, age)
        yop = padArrayWithZeros(shape, yop)
        pub = padArrayWithZeros(shape, pub)
        atr = padArrayWithZeros(shape, atr)
        city = padArrayWithZeros(shape, city)
        state = padArrayWithZeros(shape, state)
        print
        output = padArrayWithZeros(shape, output)
        print('Readjusted: age{}, yop{}, atr{}, city{}, state{}, output{}'.format(age.shape, yop.shape, atr.shape, city.shape, state.shape, output.shape))
    return {input_age: age, input_year: yop, input_author: atr, input_city: city, input_state: state, actual_y: output}

In [163]:
def padArrayWithZeros(shape, arr):
    zeroPad = np.zeros(shape)
    return np.concatenate([arr, zeroPad], axis=0)

In [164]:
%%time
with tf.Session() as session:
    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())
    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    epochWiseCost = []
    for epoch in range(1):
        # Feed in the training data and do one step of neural network training
#         session.run(train_iterator.initializer)
#         idx = 0
#         print('Epoch {}'.format(epoch))
#         while True:
#             try:
#                 batch = train_iterator.get_next()
#                 idx = idx + 1
#                 feedDict = getFeedDict(batch, batchSize, session)
#                 session.run(optimizer, feed_dict=feedDict)
#                 print('Batch {} Size: {}'.format(idx, batch[0].eval(session=session).shape[0]))
#             except tf.errors.OutOfRangeError:
#                 break
                
        session.run(test_iterator.initializer)
        idx = 0
        batchWiseCost = []
        while True:
            try:
                batch = test_iterator.get_next()
                idx = idx + 1
                feedDict = getFeedDict(batch, batchSize, session)
                testCost = session.run(cost, feed_dict=feedDict)
                print("Batch: {} Cost: {}".format(idx, testCost))
                batchWiseCost.append(testCost)
            except Exception as e: 
                print('Exception Occurred', e)
                break
        epochTestCost = np.array(batchWiseCost).mean()
        epochWiseCost.append(epochTestCost)
        print("Epoch:{} Testing Cost: {}".format(epoch, epochTestCost))

        

Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch: 1 Cost: 116.84249006052922
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch: 2 Cost: 123.58526180628435
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch: 3 Cost: 118.8998738040132
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch: 4 Cost: 117.57023004752958
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(6, 1)
Exception Occurred Incompatible shapes: [256,1] vs. [6,1]
	 [[Node: SquaredDifference = SquaredDifference[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:CPU:0"](add_3, _arg_Actual_Y_0_0)]]

Caused by op 'SquaredDi

### Session Initialization

In [165]:
%%time
with tf.Session() as session:
    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())
    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    epochWiseCost = []
    for epoch in range(10):
        # Feed in the training data and do one step of neural network training
        session.run(train_iterator.initializer)
        idx = 0
        print('Epoch {}'.format(epoch))
        while True:
            try:
                batch = train_iterator.get_next()
                idx = idx + 1
                feedDict = getFeedDict(batch, batchSize, session)
                session.run(optimizer, feed_dict=feedDict)
                print('Batch {} Size: {}'.format(idx, batch[0].eval(session=session).shape[0]))
            except tf.errors.OutOfRangeError:
                break
                
        session.run(test_iterator.initializer)
        idx = 0
        batchWiseCost = []
        while True:
            try:
                batch = test_iterator.get_next()
                idx = idx + 1
                feedDict = getFeedDict(batch, batchSize, session)
                batchWiseCost.append(session.run(cost, feed_dict=feedDict))
            except tf.errors.OutOfRangeError:
                break
        epochTestCost = np.array(batchWiseCost).mean()
        epochWiseCost.append(epochTestCost)
        print("Epoch:{} Testing Cost: {}".format(epoch, epochTestCost))

        

Epoch 0
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch 1 Size: 256
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch 2 Size: 256
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch 3 Size: 256
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch 4 Size: 256
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch 5 Size: 256
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), output(256, 1)
Batch 6 Size: 256
Required Batch Size: 256, Actual Batch Size: age(256, 1), yop(256, 1), atr(256, 1), city(256, 1), state(256, 1), out

InvalidArgumentError: Incompatible shapes: [256,1] vs. [6,1]
	 [[Node: SquaredDifference = SquaredDifference[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:CPU:0"](add_3, _arg_Actual_Y_0_0)]]

Caused by op 'SquaredDifference', defined at:
  File "/usr/local/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).s

In [166]:
# %%time
# train_cost = {};
# test_cost = {};
# with tf.Session() as session:
#     # Create log file writers to record training progress.
#     # We'll store training and testing log data separately.
#     training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
#     testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)

#     # Run the global variable initializer to initialize all variables and layers of the neural network
#     session.run(tf.global_variables_initializer())
#     # Run the optimizer over and over to train the network.
#     # One epoch is one full run through the training data set.
#     for epoch in range(training_epochs):
#         # Feed in the training data and do one step of neural network training
#         # 'Age', 'Year-Of-Publication', 'Book-Author-Enc', 'City-Enc','State-Enc'
#         feedDict = getFeedDict(X_train, y_train, True)
#         session.run(optimizer, feed_dict=feedDict)
#         if epoch % 5 == 0:
#             # Get the current accuracy scores by running the "cost" operation on the training and test data sets
#             feedDict = getFeedDict(X_train, y_train, True)
#             training_cost, training_summary = session.run([cost, summary], feed_dict=feedDict)
                
#             feedDict = getFeedDict(X_test, y_test, True)
#             testing_cost, testing_summary = session.run([cost, summary], feed_dict=feedDict)
#             # Write the current training status to the log files (Which we can view with TensorBoard)
#             training_writer.add_summary(training_summary, epoch)
#             testing_writer.add_summary(testing_summary, epoch)
#             # Print the current training status to the screen
#             train_cost[epoch] = training_cost;
#             test_cost[epoch] = testing_cost;
#             print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))
#     feedDict = getFeedDict(X_test, y_test, True)
#     testing_cost, training_summary = session.run([cost, summary], feed_dict=feedDict)
#     print("Testing Cost: {}".format(testing_cost))

        

In [167]:
# ! tensorboard --logdir "./logs"

In [168]:
print('[TEST] Batch Size: {} Total Batch: {} Last Batch Size: {}' . format(batchSize, y_test.shape[0] // batchSize + 1, y_test.shape[0] % batchSize))

[TEST] Batch Size: 256 Total Batch: 10 Last Batch Size: 6


In [169]:
print('[TEST] Batch Size: {} Total Batch: {} Last Batch Size: {}' . format(batchSize, y_t.shape[0] // batchSize + 1, y_test.shape[0] % batchSize))

NameError: name 'y_t' is not defined

In [ ]:
print(X_test.shape[0], X_train.shape[0])